In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

This notebook is an initial draft at coding up the functions for replicating/altering the Commodity
Trade Matters model. This is the counterfactual setup given, it is altered such that the counterfactual
scenario is looking at the impact of a supply shock. This is captured by R_hat: the endowment of 
natural resources.

In order to have a consistent framework: axis 0 will default to countries, axis 1 will default to countries
and axis 2 will default to goods (commodities or final goods).

In [2]:
syn_data = {'g':4, 'k':4, 'n':3}

In [3]:
## Expenditure

In [4]:
def get_E_hat(w_hat, r_hat, data):
    ''' This is a function that returns the total expenditure change for all countries as a vector E_hat. 
    variables: w_hat, r_hat 
    parameters: data['e_L'], data['e_R']
    output: E_hat (nx1)
    '''
    E_hat = w_hat*data['e_L'] + (r_hat * data['e_R']) @ np.ones((data['g'], 1))
    return E_hat

In [5]:
syn_data['e_L'] = np.ones((3,1))*.5
syn_data['e_R'] = np.ones((3, 4))*2
w_hat = np.ones((3,1))
r_hat = np.ones((3, 4))

In [6]:
get_E_hat(w_hat, r_hat, syn_data)

array([[8.5],
       [8.5],
       [8.5]])

In [7]:
# Seems right

In [8]:
## Price index

In [9]:
def get_P_k_hat(P_k_goods_hat, data):
    ''' This is a function that returns the price index change for a country as a vector P_k_hat.
    variables: P_k_goods_hat
    parameters: data['alpha'], data['sigma']
    output: P_k_hat (nx1)
    '''
    new_matrix = data['alpha'] * (P_k_goods_hat)**(1-data['sigma']) 
    P_k_hat = (new_matrix @ np.ones((data['k'], 1)) )**(1/(1-data['sigma']))
    
    return P_k_hat

In [10]:
syn_data['sigma'] = 2 
syn_data['alpha']=np.ones((3, 4))*2

P_k_goods_hat = np.ones((3, 4))*2

In [11]:
get_P_k_hat(P_k_goods_hat, syn_data)

array([[0.25],
       [0.25],
       [0.25]])

In [12]:
# Seems right

In [13]:
def get_P_k_goods_hat(C_k_hat, data):
    ''' This is a function that returns the price index for final goods change for a country and final good matrix.
    variables: C_k_hat
    parameters: data['lambda_k'], data['theta_k']
    output: P_k_goods_hat (nxk)
    '''
    
    P_k_goods_hat = np.zeros((data['n'], data['k']))
    C_k_hat_temp = C_k_hat ** (-data['theta_k'].reshape((1, data['k'])))
    part1 = np.sum(data['lambda_k'] * C_k_hat_temp.reshape((1, data['n'], data['k'])), axis=1)
    part2 = part1.reshape((data['n'], data['k']))
    P_k_goods_hat = part2 ** (-1/data['theta_k'].reshape((1, data['k'])))

    return P_k_goods_hat

In [14]:
syn_data['lambda_k'] = np.ones((3, 3, 4))*2
syn_data['theta_k'] = np.linspace(1, 4, 4).reshape((4,1)) 

C_k_hat = np.ones((3, 4))*4

In [15]:
P_k_goods_hat = get_P_k_goods_hat(C_k_hat, syn_data)

In [16]:
P_k_goods_hat

array([[0.66666667, 1.63299316, 2.20128483, 2.55577242],
       [0.66666667, 1.63299316, 2.20128483, 2.55577242],
       [0.66666667, 1.63299316, 2.20128483, 2.55577242]])

In [17]:
get_P_k_hat(P_k_goods_hat, syn_data)

array([[0.16903749],
       [0.16903749],
       [0.16903749]])

In [21]:
def get_P_g_goods_hat(C_g_hat, data):
    ''' This is a function that returns the price index for change inn commodities for a country and commodity matrix.
    variables: C_g_hat
    parameters: data['lambda_g'], data['theta_g']
    output: P_g_goods_hat (nxg)
    '''
        
    P_g_goods_hat = np.zeros((data['n'], data['g']))
    C_g_hat_temp = C_g_hat ** (-data['theta_g'].reshape((1, data['g'])))
    part1 = np.sum(data['lambda_g'] * C_g_hat_temp.reshape((1, data['n'], data['g'])), axis=1)
    part2 = part1.reshape((data['n'], data['g']))
    P_g_goods_hat = part2 ** (-1/data['theta_g'].reshape((1, data['g'])))

    return P_g_goods_hat

In [22]:
syn_data['lambda_g'] = np.ones((3, 3, 4))
syn_data['theta_g'] =  np.linspace(1, 4, 4).reshape((4,1))*2

C_g_hat = np.ones((3, 4))*4

P_g_goods_hat = get_P_g_goods_hat(C_g_hat, syn_data)

In [23]:
P_g_goods_hat

array([[2.30940108, 3.03934274, 3.33073271, 3.48674217],
       [2.30940108, 3.03934274, 3.33073271, 3.48674217],
       [2.30940108, 3.03934274, 3.33073271, 3.48674217]])

In [24]:
## Demand 

In [25]:
def get_D_k_hat(E_hat, P_k_goods_hat, P_k_hat, data):
    ''' This is a function that returns the demand for final goods for a country.
    variables: E_hat, P_k_goods_hat, P_k_hat
    parameters: data['sigma']
    output: D_k_hat (nxk)'''
    
    D_k_hat = E_hat * (P_k_goods_hat/P_k_hat) ** (1 - data['sigma'])
    
    return D_k_hat

In [41]:
syn_data['sigma'] = 1
E_hat = np.ones((syn_data['n'], 1))*1
P_k_goods_hat = np.ones((syn_data['n'], syn_data['k']))
P_k_hat = np.ones((syn_data['n'], 1))*2

D_k_hat = get_D_k_hat(E_hat, P_k_goods_hat, P_k_hat, syn_data)

In [42]:
# TODO: Check if they sort of work

In [43]:
def get_D_g_hat(P_g_goods_hat, C_k_hat, Y_k_hat, data):
    ''' This is a function that returns the demand for commodities for a country.
    variables: P_g_goods_hat, C_k_hat, Y_k_hat
    parameters: data['d'], data['eta']
    output: D_c_hat (nxg)
    
    TODO: Problem with reshape
    
    '''
    C_k_hat_temp = C_k_hat **(data['eta'].reshape(1, data['k']) - 1)
    
    part_1 = data['d'] * ( C_k_hat_temp * Y_k_hat ).reshape((data['n'], data['k'], 1))
    part_2 = P_g_goods_hat.reshape((data['n'], 1, data['g'])) **(1 - data['eta'].reshape((1, data['k'], 1)))
    D_g_hat = np.sum(part_1*part_2, axis=1).reshape((data['n'], data['g']))
    
    return D_g_hat
    

In [44]:
syn_data['d'] = np.ones((3, 4, 4))*2
syn_data['eta'] = np.ones((syn_data['k'], 1))*4
Y_k_hat = np.ones((syn_data['n'], syn_data['k']))
D_g_hat = get_D_g_hat(P_g_goods_hat, C_k_hat, Y_k_hat, syn_data)

In [45]:
## Production
# TODO: Check if they work

In [46]:
def get_Y_k_hat(C_k_hat, P_k_hat, D_k_hat, data):
    ''' This is a function that returns the total production for a given final good and country.
    variables: C_k_hat, P_k_hat, D_k_hat
    parameters: data['X_k'], data['Y_k'], data['theta_k']
    output: Y_k_hat (nxk)'''
    
    C_k_hat_temp = C_k_hat **(-data['theta_k'].reshape((1, data['k'])))
    P_k_hat_temp = P_k_hat **(data['theta_k'].reshape((1, data['k'])))
    
    part_1 = data['X_k'] * (P_k_hat_temp * D_k_hat).reshape((data['n'],1,data['k']))
    
    Y_k_hat = C_k_hat_temp/data['Y_k'] * np.sum(part_1, axis=1).reshape(data['n'], data['k'])
    
    return Y_k_hat

In [49]:
syn_data['X_k'] = np.ones((3, 3, 4))*2
syn_data['Y_k'] = np.ones((3, 4))*2
Y_k_hat = get_Y_k_hat(C_k_hat, P_k_hat, D_k_hat, syn_data)

In [50]:
Y_k_hat

array([[1.5   , 0.75  , 0.375 , 0.1875],
       [1.5   , 0.75  , 0.375 , 0.1875],
       [1.5   , 0.75  , 0.375 , 0.1875]])

In [55]:
def get_Y_g_hat(C_g_hat, P_g_goods_hat, D_g_hat, data):    
    ''' This is a function that returns the total production for a given final good and country.
    variables: C_g_hat, P_g_goods_hat, D_g_hat
    parameters: data['X_g'], data['Y_g'], data['theta_g']'''
    
    C_g_hat_temp = C_g_hat **(-data['theta_g'].reshape((1, data['g'])))
    P_g_goods_hat_temp = P_g_goods_hat **(data['theta_g'].reshape((1, data['g'])))
    
    part_1 = data['X_g'] * (P_g_goods_hat_temp * D_g_hat).reshape((data['n'],1,data['g']))
    
    Y_g_hat = C_g_hat_temp/data['Y_g'] * np.sum(part_1, axis=1).reshape(data['n'], data['g'])
    
    return Y_g_hat

In [57]:
syn_data['X_g'] = np.ones((3, 3, 4))*2
syn_data['Y_g'] = np.ones((3, 4))*2
Y_g_hat =  get_Y_g_hat(C_g_hat, P_g_goods_hat, D_g_hat, syn_data)

In [58]:
Y_g_hat

array([[41.56921938, 18.23605646, 13.85640646, 12.07842919],
       [41.56921938, 18.23605646, 13.85640646, 12.07842919],
       [41.56921938, 18.23605646, 13.85640646, 12.07842919]])

In [47]:
## Cost of production
# TODO: Check if they are working

In [68]:
def get_C_k_hat(w_hat, P_g_goods_hat, data):
    ''' This is a function that returns the cost of final good in a country.
    variables: w_hat, P_g_goods_hat
    parameters: data['phi_L_k'], data['phi'], data['eta'] '''
    
    part1 = data['phi_L_k'] * (w_hat.reshape((data['n'], 1)) ** (1 - data['eta'].reshape((1, data['k']))))
    part2 = data['phi'] * ((P_g_goods_hat.reshape(data['n'], 1, data['k'])) ** (1 - data['eta'].reshape((1, data['k'],1))) )
    
    C_k_hat = (part1 + np.sum(part2, axis=2).reshape((data['n'], data['k']))) ** (1/(1-data['eta'].reshape((1, data['k'])) ))
    
    return C_k_hat

In [69]:
syn_data['phi_L_k'] = np.ones((syn_data['n'], syn_data['k']))
syn_data['phi'] = np.ones((syn_data['n'], syn_data['k'],syn_data['g']))
C_k_hat = get_C_k_hat(w_hat, P_g_goods_hat, syn_data)

In [70]:
C_k_hat

array([[0.94969872, 0.94969872, 0.94969872, 0.94969872],
       [0.94969872, 0.94969872, 0.94969872, 0.94969872],
       [0.94969872, 0.94969872, 0.94969872, 0.94969872]])

In [81]:
def get_C_g_hat(w_hat, r_hat, data):
    ''' This is a function that returns the cost of Commodities in a country.
    variables: w_hat, r_hat
    parameters: data['phi_L_g'], data['phi_R'], data['rho_g']
    '''
    
    part1 = data['phi_R'] * (r_hat ** (1- data['rho_g'].reshape((1, data['g']))))
    part2 = data['phi_L_g'] * (w_hat.reshape((data['n'], 1)) ** (1 - data['rho_g'].reshape((1, data['g']))))
    
    C_g_hat = (part1 + part2) ** (1/(1-data['rho_g'].reshape((1, data['g'])) ))
    
    return C_g_hat

In [82]:
syn_data['phi_R'] = np.ones((syn_data['n'],syn_data['g']))
syn_data['phi_L_g'] = np.ones((syn_data['n'], syn_data['g']))
syn_data['rho_g'] = np.ones((syn_data['g'], 1))*0.4
C_g_hat = get_C_g_hat(w_hat, r_hat, syn_data)

In [83]:
C_g_hat

array([[3.1748021, 3.1748021, 3.1748021, 3.1748021],
       [3.1748021, 3.1748021, 3.1748021, 3.1748021],
       [3.1748021, 3.1748021, 3.1748021, 3.1748021]])

In [84]:
## Cost of production
# TODO: Write the equations out
# TODO: Check if they are working

In [85]:
def get_r_hat(Y_g_hat, C_g_hat, data):
    ''' This is a function that returns the ...
    variables: Y_g_hat, C_g_hat
    parameters: data['R_hat'], data['rho_g']'''
    
    part1 = (Y_g_hat/data['R_hat']) ** (1/data['rho_g'].reshape((1, data['g'])))
    part2 = (C_g_hat) ** (1 - 1/data['rho_g'].reshape((1, data['g'])))

    r_hat = part1*part2
    
    return r_hat

In [89]:
syn_data['R_hat'] = np.ones((syn_data['n'], syn_data['g']))
r_hat = get_r_hat(Y_g_hat, C_g_hat, syn_data)

In [90]:
r_hat

array([[1969.49409721,  251.04543635,  126.34310524,   89.62953734],
       [1969.49409721,  251.04543635,  126.34310524,   89.62953734],
       [1969.49409721,  251.04543635,  126.34310524,   89.62953734]])

In [49]:
def get_w_hat(C_g_hat, C_k_hat, Y_g_hat, Y_k_hat, data):
    ''' This is a function that returns change in wage in a given country
    variables: C_g_hat, Y_g_hat, C_k_hat, Y_k_hat
    parameters: data['phi_L_g], data['phi_L_k], data['Y_c'], data['Y_'], data['']'''
    
    '''I will not define this here as it has many w_hats in the equation itself. I can define this when I am 
    defining the equlibirium. It will be the only complicated equation I will state.'''
    return None